# FALTA FAZER:
1. criterio de selecao
2. algoritmo genetico em si

In [ ]:
# import das bibliotecas
import os
import pandas as pd
import random
from greedy_filter import *

In [ ]:
# definicao das constantes
PATH_EXCEL = os.getcwd() + '/excel_files/'

Estruturas que irao armazenar os dataframes com o histórico de preços das ações.
1. dict_excels: dicionario onde a chave é o nome da ação e o valor da chave é o dataframe
2. filenames: lista com o nome dos arquivos
3. excels: lista com os dataframes

In [ ]:
dict_excels = {}
filenames = []
excels = []
for filename in os.listdir(PATH_EXCEL):
    filenames.append(filename[:filename.find('.')])
    excels.append(pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))
    dict_excels[filename[:filename.find('.')]] = (pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))

## Modelagem do problema
### Algoritmo Genético 1 - Escolha dos Parâmetros das Regras de Filtro
1. Gene: cada um dos parâmetros do filtro
    1. x - porcentagem de variação: porcentagem acima/abaixo da ultima subida ou queda
       1. varia de 0 a 1
       2. varia de 0.01 a 0.10
    2. h - hold days: após um sinal de compra/venda esperamos por h dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    3. d - delay days: após receber um sinal ignora-se os próximos d dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    4. p - previous days: olha-se p dias para tomar uma decisão
       1. varia de 0 até o número de dias transcorridos até o momento
       2. varia de 30 até o número máximo de dias
2. Cromossomo: conjunto de parâmetros
    1. Estrutura usada: lista com os parâmetros &rarr; [x, h, d, p]
3. População: conjunto de cromossomos
    1. Tamanho da População:
4. Mutação:
    1. taxa de mutação:
5. Crossover:
    1. taxa de crossover:
6. Critério de Parada:
7. Seleção:
    1. Fitness: cálculo do lucro obtido por cada cromossomo
    2. Técnica de Seleção:
        1. Técnica 1:
        2. Técnica 2:

Funções para realizar o crossover entre dois cromossomos

In [ ]:
def crossover(chromosome1, chromosome2):
    """
        Funcao para realizar o crossover entre dois cromossomos
        :param: chromosome1 - cromossomo pai
        :param: chromosome2 - cromossomo mae
        :return: new_chromosome1 - primeiro filho gerado no crossover
        :return: new_chromosome2 - segundo filho gerado no crossover
    """
    # escolhe um gene aleatorio para realizar o crossover
    rand = random.randint(1,len(chromosome1) - 1)

    # gera os dois novos cromossomos
    new_chromosome1 = chromosome1[:rand] + chromosome2[rand:]
    new_chromosome2 = chromosome2[:rand] + chromosome1[rand:]
    
    return new_chromosome1, new_chromosome2

def crossover2(chromosome1, chromosome2):
    """
        Funcao para realizar o crossover entre dois cromossomos com 2 pontos de corte
        :param: chromosome1 - cromossomo pai
        :param: chromosome2 - cromossomo mae
        :return: new_chromosome1 - primeiro filho gerado no crossover
        :return: new_chromosome2 - segundo filho gerado no crossover
    """
    # escolhe um gene aleatorio para realizar o crossover
    rand = random.randint(1,len(chromosome1) - 2)
    rand2 = random.randint(rand+1, len(chromosome1) - 1)
    # gera os dois novos cromossomos
    new_chromosome1 = chromosome1[:rand] + chromosome2[rand:rand2] + chromosome1[rand2:]
    new_chromosome2 = chromosome2[:rand] + chromosome1[rand:rand2] + chromosome2[rand2:]
    
    return new_chromosome1, new_chromosome2

Funções para realizar a mutação de um cromossomo

In [ ]:
def mutation(chromossome, today_index, last_day_index):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :param: today_index - indice da tabela referente a data atual da acao
        :param: last_day_index - indice da tabela referente a ultima data da qual se tem informacao da acao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_position = random.randint(0,len(chromossome) - 1)
    
    # modifica o gene escolhido seguindo, como unica regra, os valores que ele pode assumir
    if gene_position == 0:
        new_chromossome[0] = random.random()
    elif gene_position == 1:
        new_chromossome[1] = random.randint(0, last_day_index - today_index)
    elif gene_position == 2:
        new_chromossome[2] = random.randint(0, last_day_index - today_index)
    elif gene_position == 3:
        new_chromossome[3] = random.randint(0, today_index)
    else:
        raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

def mutation_v2(chromossome):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_position = random.randint(0,len(chromossome) - 1)
    
    # modifica o gene escolhido seguindo, como unica regra, os valores que ele pode assumir
    if gene_position == 0:
        new_chromossome[0] = random.uniform(0.01,0.10)
    elif gene_position == 1:
        new_chromossome[1] = random.randint(1,30)
    elif gene_position == 2:
        new_chromossome[2] = random.randint(1,30)
    elif gene_position == 3:
        new_chromossome[3] = random.randint(30,500)
    else:
        raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

Função para realizar a inicialização randômica de uma população

In [ ]:
def create_population(population_size):
    """
        Funcao para criar uma populacao randomica de cromossomos
        :param: population_size - tamanho da populacao que sera criada
        :return: population - nova populacao
    """
    population = []
    for i in range(0, population_size):
        population.append([random.uniform(0,0.1), random.randint(1,30), random.randint(1,30), random.randint(1,500)])
        
    return population

In [ ]:
def fitness(np_array, chromossomes, budget):
    """
        Funcao para calcular o lucro de cada um dos cromossomos de uma dada populacao
        :param: np_array - conjunto de dados de determinada acao
        :param: chromossomes - lista de cromossomos que sera avaliada
        :param: budget - dinheiro inicial do problema
        :return: fit_chromossomes - matriz com os cromossomos e o lucro(em porcentagem) obtido por eles
    """
    fit_chromossomes = []
    for chromossome in chromossomes:
        money = greedy_filter_rule(np_array, chromossome, budget)
        fit_chromossomes.append([chromossome, (money-budget)/budget])
        
    return fit_chromossomes

In [ ]:
np_array = excels[0].values
chromossomes = create_population(10)
budget = 10000
array_with_fitness = fitness(np_array, chromossomes, budget)

In [ ]:
array_with_fitness.sort(key=lambda x: x[1], reverse = True)
print (array_with_fitness)

In [ ]:
def generate_children(old_generation, crossover_function):
    size_generation = len(old_generation)
    children = []
    for i in range (size_generation):
        for j in range (i+1, size_generation):
            new_chromossome1, new_chromossome2 = crossover_function(old_generation[i], old_generation[j])
            children.append(new_chromossome1)
            children.append(new_chromossome2)
    return children

In [ ]:
print(chromossomes[:10])
b = generate_children(chromossomes[:10], crossover2)
c = b + chromossomes[:10]

In [ ]:
def selection(stock_value, list_chromossomes, budget, cut_size):
    fitness_array = fitness(stock_value, list_chromossomes, budget)
    fitness_array.sort(key=lambda x: x[1], reverse = True)
    new_generation = []
    for i in range (0,cut_size):
        new_generation.append(fitness_array[i][0])
    return new_generation, fitness_array[:cut_size]

In [ ]:
# number_to_combine = 10 # número "n" que serão combinados 2 a 2 para gerar a nova geração
# len_next_generation = 10 # tamanho que definimos para a proxima geração
# percentage_new_generation = 0.9 # porcentagem máxima da nova geracao que será por crossover

In [ ]:
d,e = selection(np_array, c, 10000, 10)

In [ ]:
print (d)

In [ ]:
print(e)